Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

  https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Proto-Value Networks

This colab contains sample code to load and perform inference on the PVN models included in the paper.

In [ ]:
# @title Imports
import tensorflow as tf
import numpy as np

assert tf.test.gpu_device_name(), "No GPU found, change the device runtime."

In [ ]:
# @title Download PVN model
import os
import pathlib
import subprocess


game = "Freeway"  # @param ['AirRaid', 'Alien', 'Amidar', 'Assault', 'Asterix', 'Asteroids', 'Atlantis', 'BankHeist', 'BattleZone', 'BeamRider', 'Berzerk', 'Bowling', 'Boxing', 'Breakout', 'Carnival', 'Centipede', 'ChopperCommand', 'CrazyClimber', 'DemonAttack', 'DoubleDunk', 'ElevatorAction', 'Enduro', 'FishingDerby', 'Freeway', 'Frostbite', 'Gopher', 'Gravitar', 'Hero', 'IceHockey', 'Jamesbond', 'JourneyEscape', 'Kangaroo', 'Krull', 'KungFuMaster', 'MontezumaRevenge', 'MsPacman', 'NameThisGame', 'Phoenix', 'Pitfall', 'Pong', 'Pooyan', 'PrivateEye', 'Qbert', 'Riverraid', 'RoadRunner', 'Robotank', 'Seaquest', 'Skiing', 'Solaris', 'SpaceInvaders', 'StarGunner', 'Tennis', 'TimePilot', 'Tutankham', 'UpNDown', 'Venture', 'VideoPinball', 'WizardOfWor', 'YarsRevenge', 'Zaxxon']
# @markdown The name of the game in "Gym" (aka camel case) format.
# @markdown e.g., Freeway, Breakout, SpaceInvaders.
run = 1 # @param [1, 2, 3]
# @markdown Runs are 1-3 inclusively and correspond
# @markdown to the RLDS seed used for data collection.

# Create game directory
gamedir = pathlib.Path('/content') / game
gamedir.mkdir(parents=True, exist_ok=True)

command = [
    "gsutil",
    "-q",
    "-m",
    "cp",
    "-R",
    f"gs://download-dopamine-rl/pvn/{game}/{run}/*",
    gamedir.as_posix(),
]

print(f"$ {' '.join(command)}", flush=True)
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

for line in process.stdout:
  print(line.rstrip(), flush=True)

assert process.wait() == 0, "Process finished with non-zero error code"
print(f"Finished downloading {game} PVN model.")

$ gsutil -q -m cp -R gs://download-dopamine-rl/pvn/Freeway/1/* /content/Freeway
Finished downloading Freeway PVN model.


In [ ]:
# @title Reload saved model
with tf.device('/device:GPU:0'):
  pvn = tf.saved_model.load(gamedir.as_posix())

In [ ]:
# @title Perform forward pass to get PVN features
# Atari observation: [batch_size, width, height, frame_stack]
obs = np.zeros((1, 84, 84, 4))

with tf.device('/device:GPU:0'):
  features = pvn(obs).numpy()
  print(f"Shape: {features.shape!r}")
  print(features)

Shape: (1, 4096)
[[0.02910921 0.00624304 0.00605449 ... 0.142218   0.         0.        ]]
